In [4]:
import os
import importlib
import numpy as np
import torch
from PIL import Image
from wordcloud import WordCloud

os.chdir('/home/chenyun/work1/Clip')
!pwd
!echo $CUDA_VISIBLE_DEVICES
!nvidia-smi

/home/chenyun/work1/Clip
4
Sat Aug 21 16:03:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 8000     Off  | 00000000:1A:00.0 Off |                  Off |
| 37%   63C    P2   239W / 260W |  46632MiB / 48601MiB |     88%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Quadro RTX 8000     Off  | 00000000:1C:00.0 Off |     

In [5]:
from utils.clip_encoder import ClipEncoder
clip_encoder = ClipEncoder()

ClipEncoder ready.


In [7]:
import json
train_data = json.load(open('data/VisDiff/visdiff_train.json'))
test_data = json.load(open('data/VisDiff/visdiff_test.json'))
print(list(train_data.keys()))
print(list(test_data.keys()))
print(len(train_data['annotations']))
print(len(test_data['annotations']))
print(test_data['annotations'][0])

['info', 'img_all_id', 'annotations']
['info', 'img_all_id', 'annotations']
4700
2350
{'img2_id': '0905533', 'img1_id': '0715395', 'sentences2': ['flying in the air', 'stabilizer bottom of the tail', 'facing left', 'small size', 'propeller engine'], 'id': 0, 'sentences1': ['on the ground', 'stabilizer top of the tail', 'facing right', 'medium size', 'turbo fan engine']}


In [9]:
cat_phrases = dict()
for img_name in dataset.img_splits['train']:
    img_data = dataset.img_data_dict[img_name]
    if img_data['category'] not in cat_phrases:
        cat_phrases[img_data['category']] = np.zeros(656)
    for p_id in img_data['phrase_ids']:
        if p_id >= 0:
            cat_phrases[img_data['category']][p_id] += 1


In [53]:
ave_phrases = np.zeros(656)
for p_count in cat_phrases.values():
    ave_phrases += p_count
ave_phrases /= len(cat_phrases)

for p_num in [1, 5, 10, 15, 20, 25, 30, 35, 40]:
    cat_desc = dict()
    for cat, p_count in cat_phrases.items():
        p_scores = (p_count - ave_phrases) / len(dataset.img_splits['train'])
        pi_sorted = np.argsort(p_scores)
        phrases = list()
        for i in range(p_num):
            pi = pi_sorted[-i - 1]
            if p_scores[pi] < 0:
                print('WARN: score < 0: %s, %d' % (cat, pi))
                break
            phrases.append(dataset.phid_to_phrase(pi))
        cat_desc[cat] = 'An image of %s texture.' % ', '.join(phrases)
    
#     print(cat_desc)
    categories = sorted(list(cat_desc.keys()))
    cat_to_ids = {c: i for i, c in enumerate(categories)}
    text_inputs = [cat_desc[c] for c in categories]
    text_feats = clip_encoder.encode_text_list(text_inputs)
    correct = 0
    for img_name in dataset.img_splits['test']:
        img = dataset.load_img(img_name)
        img_feat = clip_encoder.encode_img(img)
        score = img_feat @ text_feats.T
        pred_ci = int(torch.argmax(score))
        if pred_ci == cat_to_ids[img_name.split('/')[0]]:
            correct += 1
    acc = correct / len(dataset.img_splits['test'])
    print(p_num, acc)

1 0.3986587183308495
5 0.485096870342772
10 0.5007451564828614
15 0.5253353204172876
20 0.5484351713859911
25 0.5320417287630402


RuntimeError: Input An image of blotchy, splotchy, stained, green, smeared, mottled, painting, spotted, grungy, spotty, multi color, brown, marbled, veined, bleached, purple, soft texture, scratched, painted, patchy, speckled, marked, freckled, dappled, abstract, messy, splattered, botany, bright, patched texture. is too long for context length 77

In [54]:
texts = ['An image of %s texture.' % c for c in categories]
text_feats = clip_encoder.encode_text_list(texts)
correct = 0
for img_name in dataset.img_splits['test']:
    img = dataset.load_img(img_name)
    img_feat = clip_encoder.encode_img(img)
    score = img_feat @ text_feats.T
    pred_ci = int(torch.argmax(score))
    if pred_ci == cat_to_ids[img_name.split('/')[0]]:
        correct += 1
acc = correct / len(dataset.img_splits['test'])

In [55]:
print(acc)

0.4105812220566319
